<a href="https://colab.research.google.com/github/prupat/interactivedataviz/blob/main/tokenization_lab2_Assignment_Prudence_Brou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You will be using the `datasets` package. You can [install the package](https://pypi.org/project/datasets/) with `$ pip install datasets`. If you do not have `pip` or `conda` installed on your machine, please install it now.

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
import nltk
import timeit
from datasets import load_dataset

The next cell is just downloading the dataset. You need to be connected to the internet for this to work.

This dataset is hosted by [Hugging Face](https://huggingface.co). Hugging Face hosts machine learning models, datasets, and more. We will reference them again. It's a great place to find corpora.


The dataset is called [American Stories](https://huggingface.co/datasets/dell-research-harvard/AmericanStories). Please skim the Dataset Card. All models and datasets on the Hugging Face hub have these associated cards.

In [9]:
# Decide what year you want between 1810 and 1963

my_year = "1961"

# Decide how many articles you want to work with
num_articles = 5

#  Download data for your choice of year (1810 to 1963)
dataset = load_dataset("dell-research-harvard/AmericanStories",
    "subset_years",
    year_list=[my_year]
)

# Get the first n articles from that year
# instantiate the counter
i=0
# instantiate the string
my_articles = ''
# loop through each article for that year
for article in dataset[my_year]:
    #the article is a dictionary,
    #we're getting the text of the article by accessing the key, "article"
    my_articles += article.get('article')
    #add one to our counter
    i+=1
    #if the counter is greater than num_articles-1, stop looping
    if i>(num_articles-1): break

#validate that it is what we expect by checking on first 100 characters
print(my_articles[:1000])


Only taking a subset of years. Change name to 'all_years' to use all years in the dataset.
{'1961': 'https://huggingface.co/datasets/dell-research-harvard/AmericanStories/resolve/main/faro_1961.tar.gz'}


Generating 1961 split: 0 examples [00:00, ? examples/s]

Loading associated
Ground-breaking ceremonies
for a new gymnaSium-science
building were held yesterday
as part of 50th anniversary
proceedings at the Maret
School, 5000 Cathedral avenue
NV.


The building is scheduled to
have the gym and an audi
torium on the first floor. with
music and science facilities on
the second floor. Attending
the ceremonies with the princi-
pal, Mrs. Edward Williams,
were K. w. Heberton, president
of the Parents Club, Mrs. John
L. Laskey, president Of the
board of trustees and Mrs.
Arthur Lowe, president of the
alumni association.will steer down at is m.p.h.
to a target-landing in front
of the spectators. The per-
formers are Navy volunteers
who ordinarily work at the
parachute test facility at El
Centre, Calf.


Other demonstrations and
displays by other services will
be included in the show.general public. Free parking
will be provided for $4,000 cars
at the air base, II miles from
Washington out the Suitland
parkway. The gates open at
1890 am, May Is, and 

In [10]:
#remove new line and other formatting characters
for char in ["\n", "\r", "\d", "\t"]:
    my_articles = my_articles.replace(char, " ")
my_articles[:1000]

'Ground-breaking ceremonies for a new gymnaSium-science building were held yesterday as part of 50th anniversary proceedings at the Maret School, 5000 Cathedral avenue NV.   The building is scheduled to have the gym and an audi torium on the first floor. with music and science facilities on the second floor. Attending the ceremonies with the princi- pal, Mrs. Edward Williams, were K. w. Heberton, president of the Parents Club, Mrs. John L. Laskey, president Of the board of trustees and Mrs. Arthur Lowe, president of the alumni association.will steer down at is m.p.h. to a target-landing in front of the spectators. The per- formers are Navy volunteers who ordinarily work at the parachute test facility at El Centre, Calf.   Other demonstrations and displays by other services will be included in the show.general public. Free parking will be provided for $4,000 cars at the air base, II miles from Washington out the Suitland parkway. The gates open at 1890 am, May Is, and at noon, May IA.  

# Whitespace tokenization


In [11]:
%%time
#this is a magic function to determine how long a cell takes to run.
#It MUST be the first thing in a cell

#split the whole string on spaces. This returns a list
whitespace_tokens = my_articles.split(' ')

#check the list
whitespace_tokens[:20]

CPU times: user 94 µs, sys: 0 ns, total: 94 µs
Wall time: 110 µs


['Ground-breaking',
 'ceremonies',
 'for',
 'a',
 'new',
 'gymnaSium-science',
 'building',
 'were',
 'held',
 'yesterday',
 'as',
 'part',
 'of',
 '50th',
 'anniversary',
 'proceedings',
 'at',
 'the',
 'Maret',
 'School,']

Note: "µs" is microseconds, or a millionth of a second 1/1,000,000

# Morphological Tokenization

Lemmatizing is the process of breaking down text into tokens by first breaking it up into "words" and then using syntactic knowledge of the language (in this case, English) to break up the words.

Princeton maintains the [morphy project](https://wordnet.princeton.edu/documentation/morphy7wn#:~:text=Morphology%20in%20WordNet%20uses%20two,word%20that%20is%20in%20WordNet.), which powers `nltk`'s [WordNet Lemmatizer](https://www.nltk.org/api/nltk.stem.wordnet.html). You do NOT need to read this entire documentation, just acknowledge that it requires a significant amount of knowledge about English in order to make it work.

In [14]:
#This lemmatizer is based on the Morphy project above
from nltk.stem import WordNetLemmatizer

#Uncomment these two lines - you may need to download these, maybe not.
#nltk.download('wordnet')
#nltk.download('omw-1.4')
wn_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
%%time

#first we have to split the string on spaces to get "words"
whitespace_tokens = my_articles.split(' ')

my_lemmas = []
for word in whitespace_tokens:
    w = wn_lemmatizer.lemmatize(word)
    my_lemmas.append(w)
my_lemmas[:20]

CPU times: user 2.31 s, sys: 219 ms, total: 2.53 s
Wall time: 2.6 s


['Ground-breaking',
 'ceremony',
 'for',
 'a',
 'new',
 'gymnaSium-science',
 'building',
 'were',
 'held',
 'yesterday',
 'a',
 'part',
 'of',
 '50th',
 'anniversary',
 'proceeding',
 'at',
 'the',
 'Maret',
 'School,']

Notice how much time it takes to tokenize on whitespace versus using morphological rules. Also notice if it produced the output you expected. Sometimes it doesn't.

ms is a millisecond, or one one thousandth of a second 1/1,000

# Byte Pair Encoding

There are two implementations of BPE here. The first [uses a package (bpe)](https://github.com/soaxelbrooke/python-bpe) that you will have to install using `pip` (see above).

This will implement the algorithm we covered in class and that you can review at [Hugging Face](https://youtu.be/HEikzVL-lZU).

In [32]:
pip install bpe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.3/421.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.9 MB/s eta 0:00:00


In [33]:
from bpe import Encoder

In [34]:
%%time
whitespace_tokens = my_articles.split(' ')

# calling the Encoder algorithm
# we've specified 100 token vocab and 95% to be tokenized
# the other 5% is transformed into UNK
encoder = Encoder(100, pct_bpe=0.95)
encoder.fit(whitespace_tokens)

CPU times: user 8.11 ms, sys: 0 ns, total: 8.11 ms
Wall time: 10.1 ms


In [35]:
#print(encoder.tokenize(my_articles))

print(next(encoder.inverse_transform(encoder.transform([my_articles]))))

ground - breaking ceremonies for a new gymnasium - science building were held yesterday as part of __unk0th anniversary proceedings at the maret school , __unk000 cathedral avenue nv . the building is scheduled to have the gym and an audi torium on the first floor . with music and science facilities on the second floor . attending the ceremonies with the princi - pal , mrs . edward williams , were k . w . heberton , president of the parents club , mrs . __unkohn l . laskey , president of the board of trustees and mrs . arthur lowe , president of the alumni association . will steer down at is m . p . h . to a target - landing in front of the spectators . the per - formers are navy volunteers who ordinarily work at the parachute test facility at el centre , calf . other demonstrations and displays by other services will be included in the show . general public . free parking will be provided for __unk __unk , 000 cars at the air base , ii miles from washington out the suitland parkway . 